<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Importar-librerias" data-toc-modified-id="Importar-librerias-1">Importar librerias</a></span></li><li><span><a href="#Extraccion-de-datos-mediate-Selenium-y-BeautifulSoup" data-toc-modified-id="Extraccion-de-datos-mediate-Selenium-y-BeautifulSoup-2">Extraccion de datos mediate Selenium y BeautifulSoup</a></span></li><li><span><a href="#Guardar-df-como-archivo-csv" data-toc-modified-id="Guardar-df-como-archivo-csv-3">Guardar df como archivo csv</a></span></li></ul></div>

**<center><font size = "6"> Scraping Reuters Data <center>**
***
Web: https://www.reuters.com/site-search/?query=Tesla+Inc&offset=0

### Importar librerias

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd

### Extraccion de datos mediate Selenium y BeautifulSoup

In [2]:
# URL de la página de noticias de Tesla en Reuters con paginación
base_url = 'https://www.reuters.com/site-search/?query=Tesla&offset={}'

In [3]:
# Inicializar el controlador de Selenium para Chrome
driver = webdriver.Chrome()

# Lista para almacenar los datos
data_list = []

# Recorrer offsets
for offset in range(0, 4220, 20):

    url = base_url.format(offset)
    driver.get(url)

    # Espera a que un elemento esté presente en la página
    wait = WebDriverWait(driver, 120)
    element = wait.until(EC.presence_of_element_located(
        (By.CSS_SELECTOR, '[class="search-results__item__2oqiX"]')))

    # Contenido de la página una vez que esté completamente cargada
    page_source = driver.page_source

    # Parsear con BeautifulSoup
    soup = BeautifulSoup(page_source, 'html.parser')
    sections = soup.find_all(
        class_="text__text__1FZLe text__inherit-color__3208F text__inherit-font__1Y8w3 text__inherit-size__1DZJi")
    titles = soup.find_all(
        class_="text__text__1FZLe text__dark-grey__3Ml43 text__inherit-font__1Y8w3 text__inherit-size__1DZJi link__underline_on_hover__2zGL4 media-story-card__heading__eqhp9")
    dates = soup.find_all(
        class_="text__text__1FZLe text__inherit-color__3208F text__regular__2N1Xr text__extra_small__1Mw6v body__base__22dCE body__extra_small_body__3QTYe media-story-card__time__2i9EK")

    # Lista de diccionarios con los datos extraídos de esta página
    page_data = [{'Section': section.get_text(strip=True),
                  'Title': title.get_text(strip=True),
                  'Date': date.get_text(strip=True)} for section, title, date in zip(sections, titles, dates)]
    data_list.extend(page_data)

# Cierra el navegador 
driver.quit()

df = pd.DataFrame(data_list)

In [4]:
df

,Section,Title,Date
0,World at Work,UAW union embraces social media as labor talks...,"September 9, 2023"
1,Legal,Elon Musk's X Corp sues California to undo con...,"September 9, 2023"
2,Technology,Congress to hold new AI hearings as it works t...,"September 8, 2023"
3,Business,"China car sales return to growth in August, Te...","September 8, 2023"
4,Insight,Why Native American tribes struggle to tap bil...,"September 8, 2023"
...,...,...,...
3964,Analysis,"U.S. stocks surge in recovery; dollar, bond yi...","May 14, 2021"
3965,Business,Wall St extends recovery at end of volatile week,"May 14, 2021"
3966,Business,"Dollar lower after weak data, as inflation fea...","May 14, 2021"
3967,World,"Take Five: Retail, PMIs and a timely tantrum a...","May 14, 2021"


### Guardar df como archivo csv

In [5]:
nombre_archivo = "Reuters_Scraping.csv"
df.to_csv(nombre_archivo, index=False)